# Web scraping and initial cleaning.

In [22]:
import os
import re
import json
import requests
import pandas as pd
import numpy as np
import time
import schedule
import math
from datetime import datetime
import logging
from dotenv import find_dotenv, load_dotenv

In [8]:
load_dotenv(find_dotenv())
api_key = str(os.getenv("API_KEY"))
challenger_url = "https://na1.api.riotgames.com/tft/league/v1/challenger?api_key=" + api_key
puuid_url = "https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/" + api_key
match_url = "https://americas.api.riotgames.com/tft/match/v1/matches/" + api_key
league_list = []
summoner_list = []
puuid_list = []
match_list = []
header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:104.0) Gecko/20100101 Firefox/104.0",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com"
}


In [9]:
CURRENT_DIR = os.path.dirname("__file__")

In [18]:
req = requests.get(challenger_url, headers=header)
print(req)
counter = 0
if req.status_code == 200:
    data_json = json.loads(req.content)
    for entry in data_json['entries']:
        counter += 1
        print(entry['summonerName'])
        league_list.append(entry['summonerName'])
        print('Success:', counter)

<Response [403]>


In [19]:
counter = 0
for summoner in league_list:
    req = requests.get(url = "https://na1.api.riotgames.com/tft/summoner/v1/summoners/by-name/" + summoner + "?api_key=" + api_key
    , headers=header)
    if req.status_code == 200:
        data_json = json.loads(req.content)
        print(data_json['puuid'])
        counter += 1
        summoner_list.append(data_json['puuid'])
        print('Success:', counter)

In [20]:
counter = 0
for puuid in summoner_list:
    req = requests.get(url = "https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/" + puuid + "/ids?start=0&count=200&api_key=" + api_key
    , headers=header)
    if req.status_code == 200:
        data_json = json.loads(req.content)
        counter += 1
        puuid_list.extend(data_json)
        print('Success:', counter) 

In [21]:
len(puuid_list)

0

In [22]:
print(puuid_list[0])

IndexError: list index out of range

In [ ]:
match_list = []

In [ ]:
# counter = 0
# for match in puuid_list:
#     req = requests.get(url = "https://americas.api.riotgames.com/tft/match/v1/matches/" + match + "?api_key=" + api_key
#     , headers=header)
#     if req.status_code == 200:
#         data_json = json.loads(req.content)
#         counter += 1
#         match_list.append(data_json)
#         print('Success:', counter)

In [ ]:
import time

counter = 0
for match in puuid_list:
    req = requests.get(url = "https://americas.api.riotgames.com/tft/match/v1/matches/" + match + "?api_key=" + api_key
    , headers=header)
    if req.status_code == 200:
        data_json = json.loads(req.content)
        counter += 1
        match_list.append(data_json)
        print('Success:', counter)
        if(counter % 100 == 0):  # 100 is the number of counter after which it needs to sleep
            time.sleep(120)   # sleep time in seconds

In [ ]:
json_object = json.dumps(match_list, indent=1)
with open(os.path.abspath(os.path.join(CURRENT_DIR, 'data/raw/match_json.json')), mode='a', encoding='utf-8') as file:
    file.write(json_object)

In [ ]:
print(match_list[0])

In [211]:
match_dict_list = []

In [212]:
with open(os.path.abspath(os.path.join(CURRENT_DIR, 'data/raw/match_json.json')), 'r', encoding='utf-8') as json_file:
            
    json_data = json.load(json_file)
    try:
        for dic in json_data:

            for participant in dic['info']['participants']:
                
            
                participant_dict = {}
                participant_dict['match_id'] = dic['metadata']['match_id']
                participant_dict['game_length'] = dic['info']['game_length']
                participant_dict['tft_set_number'] = dic['info']['tft_set_number']
                participant_dict['game_version'] = dic['info']['game_version']
                participant_dict['time_eliminated'] = participant['time_eliminated']
                participant_dict['gold_left'] = participant['gold_left']
                participant_dict['last_round'] = participant['last_round']
                participant_dict['level'] = participant['level']
                participant_dict['placement'] = participant['placement']
                participant_dict['traits'] = participant['traits']
                participant_dict['units'] = participant['units']
                #append to list
                match_dict_list.append(participant_dict)
    except KeyError as error:
            print("error")
            print(error)

match_data = pd.DataFrame(match_dict_list, columns = participant_dict.keys())
match_data.to_csv(os.path.abspath(os.path.join(CURRENT_DIR, "data/processed/master_tft_dataset.csv")), index=False)


In [213]:
len(match_dict_list)

73264

In [214]:
match_data.head()

,match_id,game_length,tft_set_number,game_version,time_eliminated,gold_left,last_round,level,placement,traits,units
0,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,1734.635498,0,30,8,7,"[{'name': 'Set7_Cavalier', 'num_units': 2, 'st...","[{'character_id': 'TFT7_Sejuani', 'itemNames':..."
1,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,1732.625000,5,30,8,6,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Leona', 'itemNames': [..."
2,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,1512.855347,23,26,7,8,"[{'name': 'Set7_Astral', 'num_units': 5, 'styl...","[{'character_id': 'TFT7_Vladimir', 'itemNames'..."
3,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,2345.331299,5,41,9,1,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Zyra', 'itemNames': ['..."
4,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,2342.827393,20,41,9,3,"[{'name': 'Set7_Dragon', 'num_units': 1, 'styl...","[{'character_id': 'TFT7_Sett', 'itemNames': []..."


In [215]:
match_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73264 entries, 0 to 73263
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   match_id         73264 non-null  object 
 1   game_length      73264 non-null  float64
 2   tft_set_number   73264 non-null  int64  
 3   game_version     73264 non-null  object 
 4   time_eliminated  73264 non-null  float64
 5   gold_left        73264 non-null  int64  
 6   last_round       73264 non-null  int64  
 7   level            73264 non-null  int64  
 8   placement        73264 non-null  int64  
 9   traits           73264 non-null  object 
 10  units            73264 non-null  object 
dtypes: float64(2), int64(5), object(4)
memory usage: 6.1+ MB


In [216]:
match_data.tft_set_number.value_counts()

7    73096
6      160
1        8
Name: tft_set_number, dtype: int64

In [217]:
match_data.game_version.value_counts()

Version 12.19.471.6581 (Oct 03 2022/10:43:59) [PUBLIC] <Releases/12.19>    35864
Version 12.18.468.3251 (Sep 16 2022/13:11:56) [PUBLIC] <Releases/12.18>    14992
Version 12.20.474.0929 (Oct 17 2022/15:33:47) [PUBLIC] <Releases/12.20>     6432
Version 12.17.465.2427 (Sep 02 2022/14:37:50) [PUBLIC] <Releases/12.17>     4520
Version 12.20.474.8882 (Oct 20 2022/13:06:55) [PUBLIC] <Releases/12.20>     4088
Version 12.17.467.4767 (Sep 13 2022/17:31:22) [PUBLIC] <Releases/12.17>     3800
Version 12.15.458.1416 (Aug 03 2022/16:12:42) [PUBLIC] <Releases/12.15>      776
Version 12.14.456.5556 (Jul 27 2022/15:55:22) [PUBLIC] <Releases/12.14>      776
Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>      736
Version 12.16.462.4391 (Aug 22 2022/17:33:38) [PUBLIC] <Releases/12.16>      480
Version 12.13.453.3037 (Jul 11 2022/18:39:20) [PUBLIC] <Releases/12.13>      472
Version 12.9.439.0127 (May 05 2022/17:15:12) [PUBLIC] <Releases/12.9>        160
Version 12.11.446.9344 (Jun 

In [218]:
set_7_ver_12_17_465 = (match_data.game_version == 'Version 12.17.465.2427 (Sep 02 2022/14:37:50) [PUBLIC] <Releases/12.17>')
set_7_ver_12_17_467 = (match_data.game_version == 'Version 12.17.467.4767 (Sep 13 2022/17:31:22) [PUBLIC] <Releases/12.17>')
set_7_ver_12_18 = (match_data.game_version == 'Version 12.18.468.3251 (Sep 16 2022/13:11:56) [PUBLIC] <Releases/12.18>')
set_7_ver_12_19 = (match_data.game_version == 'Version 12.19.471.6581 (Oct 03 2022/10:43:59) [PUBLIC] <Releases/12.19>')
set_7_ver_12_20 = (match_data.game_version == 'Version 12.20.474.0929 (Oct 17 2022/15:33:47) [PUBLIC] <Releases/12.20>')
set_7_ver_12_20_8882 = (match_data.game_version == 'Version 12.20.474.8882 (Oct 20 2022/13:06:55) [PUBLIC] <Releases/12.20>')

In [219]:
match_data = match_data[set_7_ver_12_17_465 | set_7_ver_12_17_467 | set_7_ver_12_18 | set_7_ver_12_19 | set_7_ver_12_20 | set_7_ver_12_20_8882]
match_data.head()

,match_id,game_length,tft_set_number,game_version,time_eliminated,gold_left,last_round,level,placement,traits,units
0,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,1734.635498,0,30,8,7,"[{'name': 'Set7_Cavalier', 'num_units': 2, 'st...","[{'character_id': 'TFT7_Sejuani', 'itemNames':..."
1,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,1732.625000,5,30,8,6,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Leona', 'itemNames': [..."
2,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,1512.855347,23,26,7,8,"[{'name': 'Set7_Astral', 'num_units': 5, 'styl...","[{'character_id': 'TFT7_Vladimir', 'itemNames'..."
3,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,2345.331299,5,41,9,1,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Zyra', 'itemNames': ['..."
4,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,2342.827393,20,41,9,3,"[{'name': 'Set7_Dragon', 'num_units': 1, 'styl...","[{'character_id': 'TFT7_Sett', 'itemNames': []..."


In [220]:
match_data = match_data[match_data['tft_set_number'] == 7]
match_data.head() 

,match_id,game_length,tft_set_number,game_version,time_eliminated,gold_left,last_round,level,placement,traits,units
0,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,1734.635498,0,30,8,7,"[{'name': 'Set7_Cavalier', 'num_units': 2, 'st...","[{'character_id': 'TFT7_Sejuani', 'itemNames':..."
1,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,1732.625000,5,30,8,6,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Leona', 'itemNames': [..."
2,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,1512.855347,23,26,7,8,"[{'name': 'Set7_Astral', 'num_units': 5, 'styl...","[{'character_id': 'TFT7_Vladimir', 'itemNames'..."
3,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,2345.331299,5,41,9,1,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Zyra', 'itemNames': ['..."
4,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,2342.827393,20,41,9,3,"[{'name': 'Set7_Dragon', 'num_units': 1, 'styl...","[{'character_id': 'TFT7_Sett', 'itemNames': []..."


In [221]:
df = match_data.copy()
df['traits'][0]

[{'name': 'Set7_Cavalier',
  'num_units': 2,
  'style': 1,
  'tier_current': 1,
  'tier_total': 5},
 {'name': 'Set7_Dragon',
  'num_units': 2,
  'style': 2,
  'tier_current': 2,
  'tier_total': 6},
 {'name': 'Set7_Dragonmancer',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 4},
 {'name': 'Set7_Guild',
  'num_units': 6,
  'style': 2,
  'tier_current': 6,
  'tier_total': 8},
 {'name': 'Set7_Mirage',
  'num_units': 4,
  'style': 2,
  'tier_current': 2,
  'tier_total': 4},
 {'name': 'Set7_Shapeshifter',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 2},
 {'name': 'Set7_Swiftshot',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 4},
 {'name': 'Set7_Warrior',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 3}]

In [222]:
new_list = []

for trait in df.traits:
    combination_dict = {}
    for entry in trait:
        combination_dict[entry['name']] = entry['num_units']
    new_list.append(combination_dict)

In [223]:
new_list

[{'Set7_Cavalier': 2,
  'Set7_Dragon': 2,
  'Set7_Dragonmancer': 1,
  'Set7_Guild': 6,
  'Set7_Mirage': 4,
  'Set7_Shapeshifter': 1,
  'Set7_Swiftshot': 1,
  'Set7_Warrior': 1},
 {'Set7_Bard': 1,
  'Set7_Cavalier': 2,
  'Set7_Dragon': 2,
  'Set7_Dragonmancer': 1,
  'Set7_Guardian': 2,
  'Set7_Guild': 1,
  'Set7_Mirage': 8,
  'Set7_Mystic': 1,
  'Set7_Shimmerscale': 3,
  'Set7_Warrior': 1},
 {'Set7_Astral': 5,
  'Set7_Bruiser': 2,
  'Set7_Mage': 3,
  'Set7_Shapeshifter': 1,
  'Set7_Swiftshot': 1,
  'Set7_Warrior': 1,
  'Set7_Whispers': 2},
 {'Set7_Bard': 1,
  'Set7_Cannoneer': 2,
  'Set7_Evoker': 2,
  'Set7_Guardian': 2,
  'Set7_Guild': 1,
  'Set7_Lagoon': 3,
  'Set7_Mystic': 3,
  'Set7_Ragewing': 1,
  'Set7_Tempest': 1,
  'Set7_Warrior': 1,
  'Set7_Whispers': 2},
 {'Set7_Dragon': 1,
  'Set7_Dragonmancer': 8,
  'Set7_Guild': 1,
  'Set7_Jade': 1,
  'Set7_Lagoon': 1,
  'Set7_Mirage': 1,
  'Set7_Ragewing': 4,
  'Set7_Shapeshifter': 2,
  'Set7_Shimmerscale': 1,
  'Set7_Tempest': 1,
  'Set7_

In [224]:
df['combination'] = new_list
df.head()

,match_id,game_length,tft_set_number,game_version,time_eliminated,gold_left,last_round,level,placement,traits,units,combination
0,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,1734.635498,0,30,8,7,"[{'name': 'Set7_Cavalier', 'num_units': 2, 'st...","[{'character_id': 'TFT7_Sejuani', 'itemNames':...","{'Set7_Cavalier': 2, 'Set7_Dragon': 2, 'Set7_D..."
1,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,1732.625000,5,30,8,6,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Leona', 'itemNames': [...","{'Set7_Bard': 1, 'Set7_Cavalier': 2, 'Set7_Dra..."
2,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,1512.855347,23,26,7,8,"[{'name': 'Set7_Astral', 'num_units': 5, 'styl...","[{'character_id': 'TFT7_Vladimir', 'itemNames'...","{'Set7_Astral': 5, 'Set7_Bruiser': 2, 'Set7_Ma..."
3,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,2345.331299,5,41,9,1,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Zyra', 'itemNames': ['...","{'Set7_Bard': 1, 'Set7_Cannoneer': 2, 'Set7_Ev..."
4,NA1_4471233617,2353.634277,7,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,2342.827393,20,41,9,3,"[{'name': 'Set7_Dragon', 'num_units': 1, 'styl...","[{'character_id': 'TFT7_Sett', 'itemNames': []...","{'Set7_Dragon': 1, 'Set7_Dragonmancer': 8, 'Se..."


In [225]:
df['units'][0]

[{'character_id': 'TFT7_Sejuani',
  'itemNames': [],
  'items': [],
  'name': '',
  'rarity': 0,
  'tier': 1},
 {'character_id': 'TFT7_Twitch',
  'itemNames': [],
  'items': [],
  'name': '',
  'rarity': 1,
  'tier': 1},
 {'character_id': 'TFT7_DragonGuild',
  'itemNames': ['TFT7_Item_CavalierEmblemItem',
   'TFT_Item_RedBuff',
   'TFT_Item_RunaansHurricane'],
  'items': [2307, 57, 26],
  'name': '',
  'rarity': 3,
  'tier': 2},
 {'character_id': 'TFT7_DragonBlue',
  'itemNames': ['TFT_Item_ArchangelsStaff',
   'TFT_Item_GuinsoosRageblade',
   'TFT_Item_Bloodthirster'],
  'items': [34, 23, 16],
  'name': '',
  'rarity': 5,
  'tier': 2},
 {'character_id': 'TFT7_Jayce',
  'itemNames': ['TFT_Item_RabadonsDeathcap',
   'TFT_Item_Zephyr',
   'TFT_Item_IonicSpark'],
  'items': [33, 67, 36],
  'name': '',
  'rarity': 4,
  'tier': 2},
 {'character_id': 'TFT7_Yasuo',
  'itemNames': ['TFT_Item_Shroud'],
  'items': [59],
  'name': '',
  'rarity': 6,
  'tier': 1}]

In [226]:
new_list2 = []

for unit in df.units:
    champion_dict = {}
    for entry in unit:
        champion_dict[entry['character_id']] = dict(items = entry['itemNames'], tier = entry['tier'])
    new_list2.append(champion_dict)

In [227]:
new_list2

[{'TFT7_Sejuani': {'items': [], 'tier': 1},
  'TFT7_Twitch': {'items': [], 'tier': 1},
  'TFT7_DragonGuild': {'items': ['TFT7_Item_CavalierEmblemItem',
    'TFT_Item_RedBuff',
    'TFT_Item_RunaansHurricane'],
   'tier': 2},
  'TFT7_DragonBlue': {'items': ['TFT_Item_ArchangelsStaff',
    'TFT_Item_GuinsoosRageblade',
    'TFT_Item_Bloodthirster'],
   'tier': 2},
  'TFT7_Jayce': {'items': ['TFT_Item_RabadonsDeathcap',
    'TFT_Item_Zephyr',
    'TFT_Item_IonicSpark'],
   'tier': 2},
  'TFT7_Yasuo': {'items': ['TFT_Item_Shroud'], 'tier': 1}},
 {'TFT7_Leona': {'items': [], 'tier': 2},
  'TFT7_Nunu': {'items': ['TFT_Item_RecurveBow',
    'TFT7_Item_ShimmerscaleHeartOfGold'],
   'tier': 2},
  'TFT7_DragonGold': {'items': ['TFT_Item_DragonsClaw',
    'TFT_Item_WarmogsArmor',
    'TFT7_Item_MirageEmblemItem'],
   'tier': 1},
  'TFT7_DragonBlue': {'items': ['TFT_Item_InfinityEdge',
    'TFT_Item_GuinsoosRageblade',
    'TFT_Item_MadredsBloodrazor'],
   'tier': 2},
  'TFT7_Yasuo': {'items': ['T

In [228]:
df['champion'] = new_list2

In [229]:
df.drop(columns=['traits', 'units', 'tft_set_number'], axis=1, inplace=True)

In [230]:
df.head()

,match_id,game_length,game_version,time_eliminated,gold_left,last_round,level,placement,combination,champion
0,NA1_4471233617,2353.634277,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,1734.635498,0,30,8,7,"{'Set7_Cavalier': 2, 'Set7_Dragon': 2, 'Set7_D...","{'TFT7_Sejuani': {'items': [], 'tier': 1}, 'TF..."
1,NA1_4471233617,2353.634277,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,1732.625000,5,30,8,6,"{'Set7_Bard': 1, 'Set7_Cavalier': 2, 'Set7_Dra...","{'TFT7_Leona': {'items': [], 'tier': 2}, 'TFT7..."
2,NA1_4471233617,2353.634277,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,1512.855347,23,26,7,8,"{'Set7_Astral': 5, 'Set7_Bruiser': 2, 'Set7_Ma...",{'TFT7_Vladimir': {'items': ['TFT_Item_ThiefsG...
3,NA1_4471233617,2353.634277,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,2345.331299,5,41,9,1,"{'Set7_Bard': 1, 'Set7_Cannoneer': 2, 'Set7_Ev...",{'TFT7_Zyra': {'items': ['TFT_Item_ZekesHerald...
4,NA1_4471233617,2353.634277,Version 12.20.474.8882 (Oct 20 2022/13:06:55) ...,2342.827393,20,41,9,3,"{'Set7_Dragon': 1, 'Set7_Dragonmancer': 8, 'Se...","{'TFT7_Sett': {'items': [], 'tier': 3}, 'TFT7_..."


In [231]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69688 entries, 0 to 73263
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   match_id         69688 non-null  object 
 1   game_length      69688 non-null  float64
 2   game_version     69688 non-null  object 
 3   time_eliminated  69688 non-null  float64
 4   gold_left        69688 non-null  int64  
 5   last_round       69688 non-null  int64  
 6   level            69688 non-null  int64  
 7   placement        69688 non-null  int64  
 8   combination      69688 non-null  object 
 9   champion         69688 non-null  object 
dtypes: float64(2), int64(4), object(4)
memory usage: 7.9+ MB


In [232]:
df.match_id.nunique()

5144

In [233]:
df.to_csv('TFT_Challenger_NA_Data.csv', index=False)